In [179]:
class BD_Charts:
    import pandas as pd
    import matplotlib.pyplot as plt
    from openpyxl import load_workbook
    from openpyxl.drawing.image import Image
    from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
    import tempfile
    
    def __init__(self, input_file):
        """
        Initializes the BD_Charts with the path to the input Excel file.
        """
        self.input_file = input_file
        self.workbook = load_workbook(input_file)
        
    def plot_pie(input_file, column="Impact"):
        """
        Processes all sheets in an Excel file to count 'Yes', 'No', and 'Maybe' in the Impact column.
        Creates a pie chart of the aggregated counts and saves it to a new sheet in the Excel file.

        """
        # Read the Excel file
        excel_data = pd.ExcelFile(input_file)
    
        # Initialize counters for 'Yes', 'No', and 'Maybe', and 'Blanks'
        counts = {"Yes": 0, "No": 0, "Maybe": 0, "Blanks": 0}
    
        # Loop through each sheet
        for sheet_name in excel_data.sheet_names:
            df = excel_data.parse(sheet_name)
    
            # Check if the specified column exists
            if column in df.columns:
                # Count occurrences of each value
                sheet_counts = df[column].value_counts()
    
                # Update the total counts
                counts["Yes"] += sheet_counts.get("Yes", 0)
                counts["No"] += sheet_counts.get("No", 0)
                counts["Maybe"] += sheet_counts.get("Maybe", 0)
                counts["Blanks"] += df[column].isna().sum()
    
        # Create a pie chart
        labels = list(counts.keys())
        sizes = list(counts.values())
        colors = ["#F44336", "#4CAF50", "#FFFF00", "#607D8B"]  # Green, Red, Yellow for Yes, No, Maybe
        explode = explode = (0, 0,0,0) # Slightly explode the 'Yes' slice
    
        plt.figure(figsize=(5, 6))
        plt.pie(sizes, labels=labels, colors=colors, autopct="%1.1f%%", startangle=90, explode=explode, textprops={'fontsize': 11, 'fontweight': 'bold', 'color': 'white'})
        plt.title("Distribution of Impact Across Teams", fontsize = 16)
        plt.axis("equal")  # Equal aspect ratio ensures the pie chart is circular.
    
        # Save the chart to a temporary image file
        temp_image_path = tempfile.NamedTemporaryFile(suffix=".png", delete=False).name
        plt.savefig(temp_image_path, format="png")
        plt.close()
    
        # Load the Excel file
        workbook = load_workbook(input_file)
    
        # Add a new sheet for the summary
        if "Summary" not in workbook.sheetnames:
            workbook.create_sheet("Summary")
        summary_sheet = workbook["Summary"]
    
        # Insert the chart image into the summary sheet
        img = Image(temp_image_path)
        summary_sheet.add_image(img, "A1")  # Place the image in cell A1
    
        # Save the updated Excel file
        workbook.save(input_file)
    
        print(f"Pie chart added to the 'Summary' sheet in {input_file}.")
   
    def plot_bar(input_file, column="Assigned To"):
        """
        Processes all sheets in an Excel file to count assignments for each person 
        in the 'Assigned To' column. Creates a bar chart of the aggregated counts.
    
        """
        # Read the Excel file
        excel_data = pd.ExcelFile(input_file)
    
        # Initialize an empty dictionary to aggregate counts
        assignment_counts = {}
    
        # Loop through each sheet
        for sheet_name in excel_data.sheet_names:
            df = excel_data.parse(sheet_name)
    
            # Check if the specified column exists
            if column in df.columns:
                # Count occurrences of each person
                sheet_counts = df[column].value_counts().to_dict()
    
                # Update the total counts
                for person, count in sheet_counts.items():
                    if person in assignment_counts:
                        assignment_counts[person] += count
                    else:
                        assignment_counts[person] = count
    
        # Convert the aggregated counts to two lists for plotting
        people = list(assignment_counts.keys())
        counts = list(assignment_counts.values())
    
        # Create a bar chart
        plt.figure(figsize=(22, 6))
        plt.bar(people, counts, color="#4CAF50")  # Green bars
        plt.xlabel("Assigned To", fontsize=8)
        plt.ylabel("Total Number", fontsize=12)
        plt.title("Number of Assignments Per Person", fontsize=14)
        plt.xticks(rotation=45, ha="right", fontsize= 6)  # Rotate x-axis labels for readability
        plt.tight_layout()  # Adjust layout to prevent clipping
        
    
         # Save the chart to a temporary image file
        temp_image_path_2 = tempfile.NamedTemporaryFile(suffix=".png", delete=False).name
        plt.savefig(temp_image_path_2, format="png")
        plt.close()
    
        # Load the Excel file
        workbook = load_workbook(input_file)
    
        # Add a new sheet for the summary
        if "Summary" not in workbook.sheetnames:
            workbook.create_sheet("Summary")
        summary_sheet = workbook["Summary"]
    
        # Insert the chart image into the summary sheet
            # Insert the chart image into the summary sheet
        img = Image(temp_image_path_2)
        
        # Insert the chart image into the summary sheet
        img = Image(temp_image_path_2)
        summary_sheet.add_image(img, "I1")  # Place the image in cell A1
    
        # Save the updated Excel file
        workbook.save(input_file)
    
        print(f"Bar chart added to the 'Summary' sheet in {input_file}.")
        
    def sum_test(input_file, column="Testing Required"):
        """
        Aggregates the count of all unique values in the specified column across all sheets in an Excel file
        and creates a formatted summary table in a new sheet, starting at a defined cell with adjusted column widths.
        """
        # Read the Excel file
        excel_data = pd.ExcelFile(input_file)
    
        # Dictionary to store counts for each unique value
        total_counts = {}
    
        # Loop through each sheet
        for sheet_name in excel_data.sheet_names:
            df = excel_data.parse(sheet_name)
    
            # Check if the specified column exists
            if column in df.columns:
                # Count occurrences of each value (including blanks)
                sheet_counts = df[column].value_counts(dropna=False)
    
                # Update total counts
                for value, count in sheet_counts.items():
                    value = "Blanks" if pd.isna(value) or value == "" else value  # Treat NaN/empty as "Blanks"
                    total_counts[value] = total_counts.get(value, 0) + count
    
        # Create a DataFrame for the summary table
        summary_df = pd.DataFrame(list(total_counts.items()), columns=["Testing Required", "Count"])
    
        # Load the workbook
        workbook = load_workbook(input_file)
    
        # Add a new sheet for the summary if it doesn't exist
        if "Summary" not in workbook.sheetnames:
            workbook.create_sheet("Summary")
        summary_sheet = workbook["Summary"]
    
        # Define the fixed starting cell (I33)
        start_row = 33  # Row 33
        start_col = 9  # Column I (I = 9th column)
    
          # Define styles
        header_font = Font(bold=True)
        header_fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")  # Grey header
        row_fill_1 = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")  # White
        row_fill_2 = PatternFill(start_color="F0F0F0", end_color="F0F0F0", fill_type="solid")  # Light grey
        
        # Write headers
        headers = ["Testing Required", "Count"]
        for col_idx, header in enumerate(headers):
            cell = summary_sheet.cell(row=start_row, column=start_col + col_idx)
            cell.value = header
            cell.font = header_font
            cell.fill = header_fill
        
        # Write data rows with alternating row colors
        for row_idx, (value, count) in enumerate(zip(summary_df["Testing Required"], summary_df["Count"])):
            row_fill = row_fill_1 if row_idx % 2 == 0 else row_fill_2
            # Write the value and count cells
            value_cell = summary_sheet.cell(row=start_row + row_idx + 1, column=start_col)
            count_cell = summary_sheet.cell(row=start_row + row_idx + 1, column=start_col + 1)
        
            value_cell.value = value
            value_cell.fill = row_fill
        
            count_cell.value = count
            count_cell.fill = row_fill
        
        # Adjust column widths
        for col_idx, header in enumerate(headers):
            max_length = len(header)  # Start with the header length
            for row_idx in range(len(summary_df)):
                value = summary_sheet.cell(row=start_row + row_idx + 1, column=start_col + col_idx).value
                if value:
                    max_length = max(max_length, len(str(value)))
            # Add some padding for better readability
            adjusted_width = max_length + 2
            summary_sheet.column_dimensions[chr(64 + start_col + col_idx)].width = adjusted_width
        
        # Save the updated Excel file
        workbook.save(input_file)
        
        print(f"Formatted summary table added to the 'Summary' sheet starting at cell I33 in {input_file}.")

In [180]:
plot_pie(input_file)
plot_bar(input_file)
sum_test(input_file)

Pie chart added to the 'Summary' sheet in IA_2024R2_Automatically Available - copy.xlsx.
Bar chart added to the 'Summary' sheet in IA_2024R2_Automatically Available - copy.xlsx.
Formatted summary table added to the 'Summary' sheet starting at cell I33 in IA_2024R2_Automatically Available - copy.xlsx.
